# Data Analysis

>Importing libraries

In [1]:
import pandas as pd
import numpy as np
import pandasql as ps
from pandasql import sqldf
import ast
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn import preprocessing
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

# Data Cleaning & Transformation

>Importing and viewing the fashion dataset


In [2]:
fashion_dataset= pd.read_csv(r".\fashion dataset.csv")
fashion_dataset

,p_id,name,price,colour,brand,ratingCount,avg_rating,description,p_attributes
0,1518329.0,Dupatta Bazaar White Embroidered Chiffon Dupatta,899.0,White,Dupatta Bazaar,1321.0,4.548827,White embroidered&nbsp;dupattaChiffon<br>Hand-...,"{'Occasion': 'Daily', 'Pattern': 'Embroidered'..."
1,5829334.0,Roadster Women Mustard Yellow Solid Hooded Swe...,1199.0,Mustard,Roadster,5462.0,4.313255,"Mustard yellow solid sweatshirt, has a hood, t...","{'Body Shape ID': '443,424,324', 'Body or Garm..."
2,10340119.0,Inddus Peach-Coloured & Beige Unstitched Dress...,5799.0,Peach,Inddus,145.0,4.068966,Peach-Coloured and beige woven design unstitch...,"{'Bottom Fabric': 'Cotton Blend', 'Bottom Patt..."
3,10856380.0,SASSAFRAS Women Black Parallel Trousers,1499.0,Black,SASSAFRAS,9124.0,4.147523,"Black solid woven high-rise parallel trousers,...","{'Add-Ons': 'NA', 'Body Shape ID': '424', 'Bod..."
4,12384822.0,Kotty Women Black Wide Leg High-Rise Clean Loo...,1999.0,Black,Kotty,12260.0,4.078467,"Black dark wash 4-pocket high-rise jeans, clea...","{'Add-Ons': 'NA', 'Brand Fit Name': 'NA', 'Clo..."
...,...,...,...,...,...,...,...,...,...
14324,17029604.0,The Chennai Silks Pink & Silver-Toned Floral Z...,3999.0,Pink,The Chennai Silks,NaN,NaN,<b>Design Details </b><ul><li>Pink and silver-...,"{'Better Cotton Initiative': 'Regular', 'Blous..."
14325,17600212.0,Kinder Kids Girls Blue & Green Printed Foil Pr...,2050.0,Blue,Kinder Kids,NaN,NaN,"Blue and green printed lehenga choli, has foi...","{'Blouse Closure': 'NA', 'Blouse Fabric': 'Cot..."
14326,18159266.0,KLOTTHE Women Green & Black Floral Printed Pal...,1659.0,Green,KLOTTHE,NaN,NaN,<ul> <li> Green and black woven palazzos </li>...,{'Body or Garment Size': 'To-Fit Denotes Body ...
14327,18921114.0,InWeave Women Red Printed A-Line Skirt,2399.0,Red,InWeave,NaN,NaN,"<p>Red printed A-line skirt, has drawstring cl...","{'Add-Ons': 'NA', 'Body Shape ID': '324,333,42..."


>Finding the number of unique brands in the fashion dataset

In [3]:
len(pd.unique(fashion_dataset['brand']))

1021

>Importing the brand details dataset

In [4]:
brand_details = pd.read_excel(r".\fashion brand details.xlsx")
brand_details

,brand_id,brand_name
0,1,513
1,2,109F
2,3,20Dresses
3,4,250 Designs
4,5,3Pin
...,...,...
1015,1016,Ziva Fashion
1016,1017,Zivame
1017,1018,Ziyaa
1018,1019,Zoella


>Counting the number of unique brand names in the brand details dataset

In [5]:
len(pd.unique(brand_details['brand_name']))

1020

>Finding the number of null and duplicated values in each dataset

In [6]:
brand_details.isna().sum()

brand_id      0
brand_name    0
dtype: int64

In [7]:
fashion_dataset.isna().sum()

p_id              18
name              19
price             19
colour            22
brand             24
ratingCount     7748
avg_rating      7748
description       19
p_attributes      19
dtype: int64

In [8]:
brand_details.duplicated().sum()

0

In [9]:
fashion_dataset.duplicated().sum()

59

>Testing out duplicate dropping

In [10]:
df_dupl = fashion_dataset.drop_duplicates(keep = "last")
len(pd.unique(df_dupl["brand"]))

1021

>Dropping duplicates in fashion dataset

In [11]:
fashion_dataset.drop_duplicates(inplace = True, keep = "last")
fashion_dataset

,p_id,name,price,colour,brand,ratingCount,avg_rating,description,p_attributes
0,1518329.0,Dupatta Bazaar White Embroidered Chiffon Dupatta,899.0,White,Dupatta Bazaar,1321.0,4.548827,White embroidered&nbsp;dupattaChiffon<br>Hand-...,"{'Occasion': 'Daily', 'Pattern': 'Embroidered'..."
1,5829334.0,Roadster Women Mustard Yellow Solid Hooded Swe...,1199.0,Mustard,Roadster,5462.0,4.313255,"Mustard yellow solid sweatshirt, has a hood, t...","{'Body Shape ID': '443,424,324', 'Body or Garm..."
2,10340119.0,Inddus Peach-Coloured & Beige Unstitched Dress...,5799.0,Peach,Inddus,145.0,4.068966,Peach-Coloured and beige woven design unstitch...,"{'Bottom Fabric': 'Cotton Blend', 'Bottom Patt..."
3,10856380.0,SASSAFRAS Women Black Parallel Trousers,1499.0,Black,SASSAFRAS,9124.0,4.147523,"Black solid woven high-rise parallel trousers,...","{'Add-Ons': 'NA', 'Body Shape ID': '424', 'Bod..."
4,12384822.0,Kotty Women Black Wide Leg High-Rise Clean Loo...,1999.0,Black,Kotty,12260.0,4.078467,"Black dark wash 4-pocket high-rise jeans, clea...","{'Add-Ons': 'NA', 'Brand Fit Name': 'NA', 'Clo..."
...,...,...,...,...,...,...,...,...,...
14324,17029604.0,The Chennai Silks Pink & Silver-Toned Floral Z...,3999.0,Pink,The Chennai Silks,NaN,NaN,<b>Design Details </b><ul><li>Pink and silver-...,"{'Better Cotton Initiative': 'Regular', 'Blous..."
14325,17600212.0,Kinder Kids Girls Blue & Green Printed Foil Pr...,2050.0,Blue,Kinder Kids,NaN,NaN,"Blue and green printed lehenga choli, has foi...","{'Blouse Closure': 'NA', 'Blouse Fabric': 'Cot..."
14326,18159266.0,KLOTTHE Women Green & Black Floral Printed Pal...,1659.0,Green,KLOTTHE,NaN,NaN,<ul> <li> Green and black woven palazzos </li>...,{'Body or Garment Size': 'To-Fit Denotes Body ...
14327,18921114.0,InWeave Women Red Printed A-Line Skirt,2399.0,Red,InWeave,NaN,NaN,"<p>Red printed A-line skirt, has drawstring cl...","{'Add-Ons': 'NA', 'Body Shape ID': '324,333,42..."


> Expanding the p_attributes column


In [12]:
fashion_dataset['p_attributes'] = fashion_dataset['p_attributes']
not_null = pd.notnull(fashion_dataset['p_attributes'])
fash3 = fashion_dataset[not_null]['p_attributes'].apply(lambda x: ast.literal_eval(x))
temp = pd.DataFrame([*fash3], fash3.index)
fashion_dataset = pd.concat([fashion_dataset, temp], axis=1)

>Renaming columns for use in SQL and MapReduce

In [13]:
fashion_dataset.rename(columns={'Dupatta Pattern':'Dupatta_Pattern', 'Kurta Pattern':'Kurta_Pattern','Bottom Pattern': 'Bottom_Pattern', 'Top Pattern': 'Top_Pattern','Print or Pattern Type': 'Print_Pattern_Type', 'Knit or Woven':'Knit_or_Woven', 'Weave Type':'Weave_Type', 'Weave Pattern':'Weave_Pattern','Dupatta Fabric':'Dupatta_Fabric','Bottom Fabric':'Bottom_Fabric', 'Top Fabric':'Top_Fabric','Fabric Purity':'Fabric_Purity', 'Fabric Type':'Fabric_Type','Blouse Fabric':'Blouse_Fabric', 'Better Cotton Initiative':'Better_Cotton', 'Saree Fabric':'Saree_Fabric', 'Colour Family':'Colour_Family'}, inplace=True)

In [14]:
fashion_dataset['Kurta_Pattern'].isna().sum()

13657

In [15]:
fashion_dataset['Weave_Type'].isna().sum()

8923

In [16]:
fashion_dataset['Knit_or_Woven'].isna().sum()

12353

In [17]:
fashion_dataset['Weave_Pattern'].isna().sum()

13087

>Viewing all columns of the dataset

In [18]:
print(list(fashion_dataset.columns))

['p_id', 'name', 'price', 'colour', 'brand', 'ratingCount', 'avg_rating', 'description', 'p_attributes', 'Occasion', 'Pattern', 'Print_Pattern_Type', 'Body Shape ID', 'Body or Garment Size', 'Closure', 'Fabric', 'Hemline', 'Hood', 'Length', 'Neck', 'Number of Pockets', 'Pocket', 'Sleeve Length', 'Surface Styling', 'Type', 'Wash Care', 'Bottom_Fabric', 'Bottom_Pattern', 'Dupatta Border', 'Dupatta_Fabric', 'Dupatta_Pattern', 'Kurta Fabric', 'Kurta_Pattern', 'Ornamentation', 'Set Content', 'Stitch', 'Technique', 'Add-Ons', 'Brand Fit Name', 'Character', 'Fabric 2', 'Features', 'Fit', 'Fly Type', 'Main Trend', 'Multipack Set', 'Sustainable', 'Type of Pleat', 'Waist Rise', 'Weave_Type', 'Distress', 'Effects', 'Fabric 3', 'Fade', 'Reversible', 'Shade', 'Stretch', 'Type of Distress', 'Waistband', 'Center Front Open', 'Fabric_Type', 'Lining', 'Sleeve Styling', 'Transparency', 'taxMaterial', 'Front Styling', 'Knit_or_Woven', 'Sport', 'Technology', 'Blouse Closure', 'Blouse_Fabric', 'Choli Stitc

### Filling the Fabric column

In [19]:
pd.unique(fashion_dataset['Fabric'])

array([nan, 'Cotton', 'Polyester', 'Poly Crepe', 'Polyviscose',
       'Viscose Rayon', 'Wool', 'Pure Cotton', 'Cotton Blend',
       'Poly Chiffon', 'Linen', 'Poly Georgette', 'Georgette',
       'Silk Blend', 'Fleece', 'Net', 'Modal', 'Art Silk', 'Acrylic',
       'Other', 'Polycotton', 'Nylon', 'Denim', 'Leather', 'Velvet',
       'Liva', 'Poly Silk', 'Silk', 'Pure Silk', 'PU', 'Cotton Silk',
       'Corduroy', 'Organza', 'Viscose Georgette', 'Wool Blend',
       'Chiffon', 'Organic Cotton', 'Brocade', 'Faux Fur', 'Suede',
       'Satin', 'Pure Georgette', 'Chanderi Silk', 'Crepe', 'Tencel',
       'Elastane', 'Dupion', 'Jacquard', 'Synthetic Leather', 'Voile',
       'Dupion Silk', 'Velour', 'Pashmina', 'Pure Crepe', 'Lyocell',
       'Raw Silk', 'Polyester PU Coated', 'Hemp', 'Livaeco'], dtype=object)

In [20]:
sqldf("select count(Fabric) from fashion_dataset where Fabric = 'NA'")

,count(Fabric)
0,0


In [21]:
fashion_dataset['Fabric'].isna().sum()

3882

In [22]:
pd.unique(fashion_dataset['Bottom_Fabric'])

array([nan, 'Cotton Blend', 'Viscose Rayon', 'Pure Cotton', 'Satin',
       'Polyester', 'Poly Georgette', 'Shantoon', 'Silk Georgette',
       'Poly Crepe', 'Organic Cotton', 'Poly Silk', 'Silk Blend',
       'Art Silk', 'Pure Silk', 'Pure Wool', 'Poly Chiffon', 'Net',
       'Silk Crepe', 'Silk Chiffon', 'Linen', 'NA', 'Pashmina',
       'Jute Cotton', 'Nylon', 'Velvet', 'Voile'], dtype=object)

In [23]:
fashion_dataset['Bottom_Fabric'] = fashion_dataset['Bottom_Fabric'].where(fashion_dataset['Bottom_Fabric'] != 'NA', None)

In [24]:
pd.unique(fashion_dataset['Blouse_Fabric'])

array([nan, 'Viscose Rayon', 'Organza', 'Poly Silk', 'Silk Blend',
       'Cotton', 'Linen Blend', 'Poly Georgette', 'Pure Georgette',
       'Art Silk', 'Net', 'Silk', 'Satin', 'NA', 'Pure Silk', 'Polyester',
       'Brocade', 'Tissue', 'Cotton Blend', 'Silk Cotton', 'Velvet',
       'Poly Crepe', 'Pure Cotton', 'Raw Silk', 'Polycotton',
       'Pure Chiffon', 'Jute Silk', 'Poly Chiffon', 'Liva', 'Pure Crepe',
       'Pure Linen', 'Supernet', 'Jute Cotton'], dtype=object)

In [25]:
fashion_dataset['Blouse_Fabric'] = fashion_dataset['Blouse_Fabric'].where(fashion_dataset['Blouse_Fabric'] != 'NA', None)

In [26]:
pd.unique(fashion_dataset['Top_Fabric'])

array([nan, 'Cotton Blend', 'Viscose Rayon', 'Pure Cotton', 'Polyester',
       'Liva', 'Poly Crepe', 'Organic Cotton', 'Poly Silk', 'Silk Blend',
       'Poly Georgette', 'Art Silk', 'Pure Wool', 'Net', 'Silk Chiffon',
       'Linen', 'Poly Chanderi', 'Pure Silk', 'Chanderi Cotton',
       'Chanderi Silk', 'Acrylic', 'Silk Georgette', 'Nylon', 'Velvet',
       'Poly Chiffon', 'Voile'], dtype=object)

In [27]:
pd.unique(fashion_dataset['Fabric_Type'])

array([nan, 'Georgette', 'NA', 'Denim', 'Cotton', 'Lace', 'Crepe',
       'Chiffon', 'Cotton Cambric', 'Net', 'Dobby', 'Scuba', 'Satin',
       'Liva', 'Twill', 'Linen', 'Chambray', 'Velvet', 'Jacquard',
       'Corduroy', 'Schiffli', 'Terry'], dtype=object)

In [28]:
fashion_dataset['Fabric_Type'] = fashion_dataset['Fabric_Type'].where(fashion_dataset['Fabric_Type'] != 'NA', None)

In [29]:
fashion_dataset['Fabric'] = fashion_dataset['Fabric'].combine_first(fashion_dataset['Top_Fabric'])

In [30]:
fashion_dataset['Fabric'].isna().sum()

2708

In [31]:
fashion_dataset['Fabric'] = fashion_dataset['Fabric'].combine_first(fashion_dataset['Bottom_Fabric'])

In [32]:
fashion_dataset['Fabric'].isna().sum()

2076

In [33]:
fashion_dataset['Fabric'] = fashion_dataset['Fabric'].combine_first(fashion_dataset['Fabric_Type'])

In [34]:
fashion_dataset['Fabric'].isna().sum()

2025

In [35]:
fashion_dataset['Fabric'] = fashion_dataset['Fabric'].combine_first(fashion_dataset['Blouse_Fabric'])

In [36]:
fashion_dataset['Fabric'].isna().sum()

202

In [37]:
pd.unique(fashion_dataset['Better_Cotton'])

array([nan, 'Regular', 'Better Cotton Initiative'], dtype=object)

In [38]:
sqldf("select Better_Cotton from fashion_dataset where Fabric is null and Better_Cotton in('Better Cotton Initiative','Regular')")

,Better_Cotton
0,Regular
1,Regular
2,Regular


In [39]:
fashion_dataset['New_Cotton'] = fashion_dataset['Better_Cotton'].replace(['Regular', 'Better Cotton Initiative'], ['Cotton',None])

In [40]:
fashion_dataset['Fabric'] = fashion_dataset['Fabric'].combine_first(fashion_dataset['New_Cotton'])

In [41]:
fashion_dataset['Fabric'].isna().sum()

199

In [42]:
pd.unique(fashion_dataset['Dupatta_Fabric'])
fashion_dataset['Dupatta_Fabric'] = fashion_dataset['Dupatta_Fabric'].where(fashion_dataset['Dupatta_Fabric'] != 'NA', None)

In [43]:
sqldf("select Dupatta_Fabric from fashion_dataset where Dupatta_Fabric is not null and Fabric is null")

,Dupatta_Fabric
0,Net


In [44]:
sqldf("select Weave_Pattern, Fabric from fashion_dataset")

,Weave_Pattern,Fabric
0,None,None
1,None,Cotton
2,None,Cotton Blend
3,None,None
4,None,Cotton
...,...,...
14265,None,Jute Cotton
14266,None,Cotton
14267,None,Polycotton
14268,None,Viscose Rayon


In [45]:
fashion_dataset['Fabric'] = fashion_dataset['Fabric'].combine_first(fashion_dataset['Weave_Pattern'])

In [46]:
fashion_dataset['Fabric'] = fashion_dataset['Fabric'].combine_first(fashion_dataset['Dupatta_Fabric'])

In [47]:
fashion_dataset['Fabric'].isna().sum()

197

In [48]:
sqldf("select Saree_Fabric from fashion_dataset where Saree_Fabric is not null and Fabric is null")

,Saree_Fabric
0,Poly Georgette
1,Pure Linen
2,Pure Cotton
3,Cotton Blend
4,Cotton Blend
5,Pure Linen
6,Pure Cotton
7,Pure Chiffon
8,Poly Chiffon
9,Pure Cotton


In [49]:
fashion_dataset['Fabric'] = fashion_dataset['Fabric'].combine_first(fashion_dataset['Saree_Fabric'])

In [50]:
fashion_dataset['Fabric'].isna().sum()

142

In [51]:
fashion_dataset['Fabric'].mode()

0    Cotton
Name: Fabric, dtype: object

In [52]:
fashion_dataset.fillna({'Fabric':'Cotton'}, inplace=True)
fashion_dataset['Fabric'].isna().sum()

0

In [53]:
sqldf("select count(Fabric) from fashion_dataset where Fabric = 'NA'")

,count(Fabric)
0,0


### Viewing Weave Type & Knit or Woven

In [54]:
pd.unique(fashion_dataset['Weave_Type'])

array([nan, 'Woven', 'Machine Weave', 'Knitted', 'Knitted and Woven',
       'Velvet', 'Denim', 'Handloom', 'Lace', 'Chambray', 'Corduroy'],
      dtype=object)

In [55]:
pd.unique(fashion_dataset['Weave_Type'])

array([nan, 'Woven', 'Machine Weave', 'Knitted', 'Knitted and Woven',
       'Velvet', 'Denim', 'Handloom', 'Lace', 'Chambray', 'Corduroy'],
      dtype=object)

In [56]:
pd.unique(fashion_dataset['Knit_or_Woven'])

array([nan, 'Knitted', 'Knitted and Woven', 'Woven'], dtype=object)

In [57]:
pd.unique(fashion_dataset['Weave_Pattern'])

array([nan, 'Regular', 'Jacquard', 'Brocade', 'Dobby', 'Khadi'],
      dtype=object)

### Formatting Weave Pattern  and Weave Type to fill Knit or Woven

In [58]:
fashion_dataset['Weave_Pattern_2'] = fashion_dataset['Weave_Pattern'].replace(['Regular', 'Jacquard', 'Brocade', 'Dobby', 'Khadi'], ['Woven','Woven','Woven','Woven','Woven'])

In [59]:
pd.unique(fashion_dataset['Weave_Pattern_2'])

array([nan, 'Woven'], dtype=object)

In [60]:
fashion_dataset['Weave_Type_2'] = fashion_dataset['Weave_Type'].replace(['Machine Weave',
       'Velvet', 'Denim', 'Handloom', 'Lace', 'Chambray', 'Corduroy'], ['Woven','Woven','Woven','Woven','Knitted','Woven','Woven'])

In [61]:
pd.unique(fashion_dataset['Weave_Type_2'])

array([nan, 'Woven', 'Knitted', 'Knitted and Woven'], dtype=object)

### Filling Knit_or_Woven

In [62]:
fashion_dataset['Knit_or_Woven'] = fashion_dataset['Knit_or_Woven'].combine_first(fashion_dataset['Weave_Pattern_2'])

In [63]:
fashion_dataset['Knit_or_Woven'] = fashion_dataset['Knit_or_Woven'].combine_first(fashion_dataset['Weave_Type_2'])

In [64]:
fashion_dataset['Knit_or_Woven'].isna().sum()

7006

In [65]:
pd.unique(fashion_dataset['Knit_or_Woven'])

array([nan, 'Woven', 'Knitted', 'Knitted and Woven'], dtype=object)

In [66]:
len(pd.unique(fashion_dataset['Fabric']))

75

In [67]:
fabrics_knit = sqldf("select distinct Fabric from fashion_dataset where Knit_or_Woven = 'Knitted'")
fabrics_knit

,Fabric
0,Cotton
1,Polyester
2,Denim
3,Pure Cotton
4,Cotton Blend
5,Viscose Rayon
6,Fleece
7,Acrylic
8,Linen
9,Nylon


In [68]:
fabrics_knit.isna().sum()

Fabric    0
dtype: int64

In [69]:
fabrics_weave = sqldf("select distinct Fabric from fashion_dataset where Knit_or_Woven = 'Woven'")
fabrics_weave

,Fabric
0,Cotton
1,Polyester
2,Viscose Rayon
3,Pure Cotton
4,Linen
5,Georgette
6,Denim
7,Poly Crepe
8,Modal
9,Crepe


In [70]:
fabrics_weave.isna().sum()

Fabric    0
dtype: int64

In [71]:
fabrics_both = sqldf("select distinct Fabric from fashion_dataset where Knit_or_Woven = 'Knitted and Woven'")
fabrics_both

,Fabric
0,Cotton
1,Linen
2,Pure Cotton
3,Polyester
4,Fleece
5,Viscose Rayon
6,Cotton Blend
7,Acrylic
8,Polycotton
9,Nylon


In [72]:
fabrics_both.isna().sum()

Fabric    0
dtype: int64

In [73]:
knit_n_woven = sqldf("select * from fabrics_knit where fabrics_knit.Fabric in(select fabrics_weave.Fabric from fabrics_weave)")

In [74]:
knit_n_woven = sqldf("select * from knit_n_woven where Fabric not in ('Other')")
knit_n_woven

,Fabric
0,Cotton
1,Polyester
2,Denim
3,Pure Cotton
4,Cotton Blend
5,Viscose Rayon
6,Fleece
7,Acrylic
8,Linen
9,Nylon


In [75]:
fabrics_both = sqldf("select * from fabrics_both where Fabric not in (select Fabric from knit_n_woven)")
fabrics_both

,Fabric
0,Polyester PU Coated


In [76]:
both_fabrics = ps.sqldf("select * from knit_n_woven union all select * from fabrics_both")
both_fabrics

,Fabric
0,Cotton
1,Polyester
2,Denim
3,Pure Cotton
4,Cotton Blend
5,Viscose Rayon
6,Fleece
7,Acrylic
8,Linen
9,Nylon


In [77]:
both_fabrics.loc[:,'weave'] = 'Knitted and Woven'
both_fabrics

,Fabric,weave
0,Cotton,Knitted and Woven
1,Polyester,Knitted and Woven
2,Denim,Knitted and Woven
3,Pure Cotton,Knitted and Woven
4,Cotton Blend,Knitted and Woven
5,Viscose Rayon,Knitted and Woven
6,Fleece,Knitted and Woven
7,Acrylic,Knitted and Woven
8,Linen,Knitted and Woven
9,Nylon,Knitted and Woven


In [78]:
fabrics_knit = sqldf("select * from fabrics_knit where Fabric not in (select Fabric from both_fabrics)")
fabrics_knit.loc[:,['weave']]='Knitted'

In [79]:
fabrics_knit

,Fabric,weave
0,Scuba,Knitted
1,Other,Knitted
2,Velour,Knitted


In [80]:
fabrics_weave = sqldf("select * from fabrics_weave where Fabric not in (select Fabric from both_fabrics)")
fabrics_weave.loc[:,['weave']]='Woven'

In [81]:
fabrics_weave

,Fabric,weave
0,Poly Crepe,Woven
1,Crepe,Woven
2,Poly Silk,Woven
3,Poly Georgette,Woven
4,Art Silk,Woven
5,Leather,Woven
6,Other,Woven
7,Regular,Woven
8,Net,Woven
9,Silk Chiffon,Woven


In [82]:
fabric_weave_type = sqldf("select * from fabrics_weave union all select * from fabrics_knit union all select * from both_fabrics")
fabric_weave_type

,Fabric,weave
0,Poly Crepe,Woven
1,Crepe,Woven
2,Poly Silk,Woven
3,Poly Georgette,Woven
4,Art Silk,Woven
5,Leather,Woven
6,Other,Woven
7,Regular,Woven
8,Net,Woven
9,Silk Chiffon,Woven


In [83]:
fashion_dataset = sqldf("select fashion_dataset.*, fabric_weave_type.weave from fashion_dataset left join fabric_weave_type on (fabric_weave_type.Fabric=fashion_dataset.Fabric)")

In [125]:
fashion_dataset['Knit_or_Woven'] = fashion_dataset['Knit_or_Woven'].combine_first(fashion_dataset['weave'])

In [129]:
sqldf("select weave from fabric_weave_type where Fabric = 'Other'")

,weave
0,Woven
1,Knitted


> Use 'Occasion', 'Print or Pattern Type', 'Pattern', 'Top Pattern', 'Bottom Pattern', 'Dupatta Pattern', 'Kurta Pattern', 'Weave Pattern', 'Knit or Woven', 'Sustainable', 'Sport', 'Fusion_Wear'

### Merging the datasets

In [86]:
merged_dataset = ps.sqldf("select fashion_dataset.p_id, fashion_dataset.name, fashion_dataset.price, fashion_dataset.colour,fashion_dataset.Colour_Family, brand_details.*, fashion_dataset.brand, fashion_dataset.ratingCount, fashion_dataset.avg_rating, fashion_dataset.Occasion, fashion_dataset.Print_Pattern_Type, fashion_dataset.Pattern, fashion_dataset.Top_Pattern, fashion_dataset.Bottom_Pattern, fashion_dataset.Dupatta_Pattern, fashion_dataset.Kurta_Pattern, fashion_dataset.Sustainable, fashion_dataset.Weave_Type, fashion_dataset.Knit_or_Woven,fashion_dataset.Weave_Pattern, fashion_dataset.Fabric, fashion_dataset.Fabric_Purity from fashion_dataset left join brand_details on (fashion_dataset.brand = brand_details.brand_name)")
merged_dataset

,p_id,name,price,colour,Colour_Family,brand_id,brand_name,brand,ratingCount,avg_rating,...,Top_Pattern,Bottom_Pattern,Dupatta_Pattern,Kurta_Pattern,Sustainable,Weave_Type,Knit_or_Woven,Weave_Pattern,Fabric,Fabric_Purity
0,1518329.0,Dupatta Bazaar White Embroidered Chiffon Dupatta,899.0,White,None,242.0,Dupatta Bazaar,Dupatta Bazaar,1321.0,4.548827,...,None,None,None,None,None,None,None,None,Cotton,None
1,5829334.0,Roadster Women Mustard Yellow Solid Hooded Swe...,1199.0,Mustard,None,750.0,Roadster,Roadster,5462.0,4.313255,...,None,None,None,None,None,None,None,None,Cotton,None
2,10340119.0,Inddus Peach-Coloured & Beige Unstitched Dress...,5799.0,Peach,None,389.0,Inddus,Inddus,145.0,4.068966,...,None,Woven Design,Woven Design,Woven Design,None,None,None,None,Cotton Blend,None
3,10856380.0,SASSAFRAS Women Black Parallel Trousers,1499.0,Black,None,NaN,None,SASSAFRAS,9124.0,4.147523,...,None,None,None,None,Regular,Woven,Woven,None,Cotton,None
4,12384822.0,Kotty Women Black Wide Leg High-Rise Clean Loo...,1999.0,Black,None,482.0,Kotty,Kotty,12260.0,4.078467,...,None,None,None,None,None,None,None,None,Cotton,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14315,17029604.0,The Chennai Silks Pink & Silver-Toned Floral Z...,3999.0,Pink,None,880.0,The Chennai Silks,The Chennai Silks,NaN,NaN,...,None,None,None,None,Regular,None,None,None,Jute Cotton,None
14316,17600212.0,Kinder Kids Girls Blue & Green Printed Foil Pr...,2050.0,Blue,None,471.0,Kinder Kids,Kinder Kids,NaN,NaN,...,Printed,Printed,NA,None,Regular,None,None,None,Cotton,None
14317,18159266.0,KLOTTHE Women Green & Black Floral Printed Pal...,1659.0,Green,None,NaN,None,KLOTTHE,NaN,NaN,...,None,None,None,None,Regular,Woven,Woven,None,Polycotton,None
14318,18921114.0,InWeave Women Red Printed A-Line Skirt,2399.0,Red,None,NaN,None,InWeave,NaN,NaN,...,None,None,None,None,Regular,None,Woven,None,Viscose Rayon,None


### Dropping unnecessary null values

In [87]:
merged_dataset.isna().sum()

p_id                      1
name                      2
price                     2
colour                    5
Colour_Family         13857
brand_id               6220
brand_name             6220
brand                     7
ratingCount            7744
avg_rating             7744
Occasion               1101
Print_Pattern_Type     4307
Pattern                3807
Top_Pattern           12162
Bottom_Pattern        11527
Dupatta_Pattern       12182
Kurta_Pattern         13707
Sustainable            1305
Weave_Type             8954
Knit_or_Woven          7037
Weave_Pattern         13137
Fabric                    0
Fabric_Purity         13776
dtype: int64

In [88]:
sqldf("select * from merged_dataset where p_id is null")

,p_id,name,price,colour,Colour_Family,brand_id,brand_name,brand,ratingCount,avg_rating,...,Top_Pattern,Bottom_Pattern,Dupatta_Pattern,Kurta_Pattern,Sustainable,Weave_Type,Knit_or_Woven,Weave_Pattern,Fabric,Fabric_Purity
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Cotton,None


In [89]:
merged_dataset.dropna(subset = "p_id", inplace = True)
merged_dataset.isna().sum()

p_id                      0
name                      1
price                     1
colour                    4
Colour_Family         13856
brand_id               6219
brand_name             6219
brand                     6
ratingCount            7743
avg_rating             7743
Occasion               1100
Print_Pattern_Type     4306
Pattern                3806
Top_Pattern           12161
Bottom_Pattern        11526
Dupatta_Pattern       12181
Kurta_Pattern         13706
Sustainable            1304
Weave_Type             8953
Knit_or_Woven          7036
Weave_Pattern         13136
Fabric                    0
Fabric_Purity         13775
dtype: int64

In [90]:
sqldf("select * from merged_dataset where name is null")

,p_id,name,price,colour,Colour_Family,brand_id,brand_name,brand,ratingCount,avg_rating,...,Top_Pattern,Bottom_Pattern,Dupatta_Pattern,Kurta_Pattern,Sustainable,Weave_Type,Knit_or_Woven,Weave_Pattern,Fabric,Fabric_Purity
0,19068208.0,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,Cotton,None


In [91]:
merged_dataset.dropna(subset = "name", inplace = True)
merged_dataset.isna().sum()

p_id                      0
name                      0
price                     0
colour                    3
Colour_Family         13855
brand_id               6218
brand_name             6218
brand                     5
ratingCount            7742
avg_rating             7742
Occasion               1099
Print_Pattern_Type     4305
Pattern                3805
Top_Pattern           12160
Bottom_Pattern        11525
Dupatta_Pattern       12180
Kurta_Pattern         13705
Sustainable            1303
Weave_Type             8952
Knit_or_Woven          7035
Weave_Pattern         13135
Fabric                    0
Fabric_Purity         13774
dtype: int64

>Upon inspection, the rows where brand was null were duplicate rows, and were therefore dropped.

In [92]:
merged_dataset.dropna(subset = "brand", inplace = True)
merged_dataset.isna().sum()

p_id                      0
name                      0
price                     0
colour                    3
Colour_Family         13850
brand_id               6213
brand_name             6213
brand                     0
ratingCount            7740
avg_rating             7740
Occasion               1099
Print_Pattern_Type     4305
Pattern                3805
Top_Pattern           12155
Bottom_Pattern        11520
Dupatta_Pattern       12175
Kurta_Pattern         13700
Sustainable            1302
Weave_Type             8948
Knit_or_Woven          7035
Weave_Pattern         13130
Fabric                    0
Fabric_Purity         13769
dtype: int64

>Finding why there are null brand name and brand ID values

In [93]:
sqldf("select distinct brand from merged_dataset where brand_id is null")

,brand
0,SASSAFRAS
1,KASSUALLY
2,Saree mall
3,MONTREZ
4,DOLCE CRUDO
...,...
515,SARIYA
516,kasee
517,SHIVANGI clothing
518,ARTICALE


In [94]:
sqldf("select distinct brand, brand_id from merged_dataset order by brand_id asc")

,brand,brand_id
0,SASSAFRAS,NaN
1,KASSUALLY,NaN
2,Saree mall,NaN
3,MONTREZ,NaN
4,DOLCE CRUDO,NaN
...,...,...
1015,Zigo,1011.0
1016,Zima Leto,1013.0
1017,Zink London,1014.0
1018,Ziva Fashion,1016.0


In [95]:
sqldf("select distinct brand_id from merged_dataset order by brand_id asc")

,brand_id
0,NaN
1,1.0
2,2.0
3,3.0
4,7.0
...,...
496,1011.0
497,1013.0
498,1014.0
499,1016.0


In [96]:
sqldf("select brand_id from brand_details where brand_name = 'KASSUALLY'")

,brand_id


In [97]:
sqldf("select distinct * from merged_dataset where brand is null")

,p_id,name,price,colour,Colour_Family,brand_id,brand_name,brand,ratingCount,avg_rating,...,Top_Pattern,Bottom_Pattern,Dupatta_Pattern,Kurta_Pattern,Sustainable,Weave_Type,Knit_or_Woven,Weave_Pattern,Fabric,Fabric_Purity


In [98]:
sqldf("select * from merged_dataset where brand_name is null")

,p_id,name,price,colour,Colour_Family,brand_id,brand_name,brand,ratingCount,avg_rating,...,Top_Pattern,Bottom_Pattern,Dupatta_Pattern,Kurta_Pattern,Sustainable,Weave_Type,Knit_or_Woven,Weave_Pattern,Fabric,Fabric_Purity
0,10856380.0,SASSAFRAS Women Black Parallel Trousers,1499.0,Black,None,None,None,SASSAFRAS,9124.0,4.147523,...,None,None,None,None,Regular,Woven,Woven,None,Cotton,None
1,12742100.0,KASSUALLY Women Black & Pink Printed Basic Jum...,2199.0,Black,None,None,None,KASSUALLY,6297.0,4.349214,...,None,None,None,None,Regular,None,None,None,Polyester,None
2,13842966.0,Sassafras Brown & Red Geometric Printed George...,1499.0,Brown,None,None,None,SASSAFRAS,7358.0,4.395352,...,None,None,None,None,Regular,Woven,Woven,None,Polyester,None
3,16595858.0,Saree Mall Floral Saree,3599.0,Pink,None,None,None,Saree mall,1005.0,3.980100,...,None,None,None,None,Regular,None,None,None,Organza,None
4,18601482.0,MONTREZ Women White Black Open Front Jacket,1999.0,White,None,None,None,MONTREZ,61.0,4.377049,...,None,None,None,None,Regular,Woven,Woven,None,Cotton,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,18055840.0,tantkatha Black Front Closure Saree Blouse Wit...,1699.0,Black,None,None,None,tantkatha,NaN,NaN,...,None,None,None,None,Regular,None,None,None,Dupion,None
6209,19361072.0,BoStreet Women Green Solid Mom Fit Trousers,2599.0,Green,None,None,None,BoStreet,NaN,NaN,...,None,None,None,None,Regular,Knitted,Knitted,None,Polyester,None
6210,18159266.0,KLOTTHE Women Green & Black Floral Printed Pal...,1659.0,Green,None,None,None,KLOTTHE,NaN,NaN,...,None,None,None,None,Regular,Woven,Woven,None,Polycotton,None
6211,18921114.0,InWeave Women Red Printed A-Line Skirt,2399.0,Red,None,None,None,InWeave,NaN,NaN,...,None,None,None,None,Regular,None,Woven,None,Viscose Rayon,None


In [99]:
sqldf("select count(distinct brand_id) from merged_dataset")

,count(distinct brand_id)
0,500


In [100]:
sqldf("select max(brand_id), brand_name from brand_details")

,max(brand_id),brand_name
0,1020,Zola


### Filling brand ID

>Getting rid of null values in brand and brand ID by assigning new IDs

In [101]:
new_id = sqldf("select distinct brand, brand_id from merged_dataset order by brand asc")
new_id

,brand,brand_id
0,109F,2.0
1,20Dresses,3.0
2,250 DESIGNS,NaN
3,3PIN,NaN
4,513,1.0
...,...,...
1015,trueBrowns,NaN
1016,urSense,NaN
1017,wild U,NaN
1018,zebu,NaN


In [102]:
new_id.loc[:, 'brandID'] = range(1, 1021)
new_id.drop('brand_id', axis = 'columns')

,brand,brandID
0,109F,1
1,20Dresses,2
2,250 DESIGNS,3
3,3PIN,4
4,513,5
...,...,...
1015,trueBrowns,1016
1016,urSense,1017
1017,wild U,1018
1018,zebu,1019


In [103]:
dataset_final = sqldf("select merged_dataset.*, new_id.brandID from merged_dataset left join new_id on (merged_dataset.brand = new_id.brand)")
dataset_final.drop(['brand_id','brand_name'], axis = 'columns', inplace = True)
dataset_final.isna().sum()

p_id                      0
name                      0
price                     0
colour                    3
Colour_Family         13850
brand                     0
ratingCount            7740
avg_rating             7740
Occasion               1099
Print_Pattern_Type     4305
Pattern                3805
Top_Pattern           12155
Bottom_Pattern        11520
Dupatta_Pattern       12175
Kurta_Pattern         13700
Sustainable            1302
Weave_Type             8948
Knit_or_Woven          7035
Weave_Pattern         13130
Fabric                    0
Fabric_Purity         13769
brandID                   0
dtype: int64

### Filling Print_Pattern_Type null values

In [104]:
pd.unique(dataset_final['Print_Pattern_Type'])

array(['Floral', 'Solid', 'Ethnic Motifs', None, 'Geometric', 'Graphic',
       'Self Design', 'Checked', 'Typography', 'Washed', 'Kalamkari',
       'Embellished', 'Humour and Comic', 'Animal', 'Paisley', 'Ribbed',
       'Striped', 'Bandhani', 'Abstract', 'Camouflage', 'Tie and Dye',
       'Woven Design', 'Colourblocked', 'Vertical Stripes', 'Polka Dots',
       'Leheriya', 'Cable Knit', 'Brand Logo', 'Open Knit',
       'Alphanumeric', 'Other', 'Conversational', 'NA', 'Chevron',
       'Ombre', 'Horizontal Stripes', 'Textured', 'Superhero', 'Warli',
       'Boucle', 'Batik', 'Quirky', 'Tribal', 'Ajrak', 'Embroidered',
       'Sequinned Stripes', 'Botanical', 'Shimmer', 'Bagh', 'Houndstooth',
       'Fair Isle', 'Dabu', 'Cartoon Characters', 'Argyle'], dtype=object)

In [105]:
sqldf("select Print_Pattern_Type, Pattern, Top_Pattern, Bottom_Pattern from dataset_final where Print_Pattern_Type is null and Top_Pattern is not null or Bottom_Pattern is not null or Pattern is not null")

,Print_Pattern_Type,Pattern,Top_Pattern,Bottom_Pattern
0,Floral,Embroidered,None,None
1,Solid,Solid,None,None
2,Ethnic Motifs,None,None,Woven Design
3,Solid,Solid,None,None
4,None,Printed,None,None
...,...,...,...,...
13296,Floral,Woven Design,None,None
13297,None,None,Printed,Printed
13298,Floral,Printed,None,None
13299,Floral,Printed,None,None


In [106]:
dataset_final['Print_Pattern_Type'] = dataset_final['Print_Pattern_Type'].combine_first(dataset_final['Pattern'])

In [107]:
dataset_final['Print_Pattern_Type'] = dataset_final['Print_Pattern_Type'].combine_first(dataset_final['Bottom_Pattern'])

In [108]:
dataset_final['Print_Pattern_Type'] = dataset_final['Print_Pattern_Type'].combine_first(dataset_final['Top_Pattern'])

In [109]:
dataset_final['Print_Pattern_Type'] = dataset_final['Print_Pattern_Type'].combine_first(dataset_final['Kurta_Pattern'])

In [110]:
dataset_final['Print_Pattern_Type'] = dataset_final['Print_Pattern_Type'].combine_first(dataset_final['Dupatta_Pattern'])

In [111]:
dataset_final.drop(['Pattern','Top_Pattern','Bottom_Pattern', 'Kurta_Pattern','Dupatta_Pattern'], axis = 'columns', inplace=True)

In [112]:
dataset_final.isna().sum()

p_id                      0
name                      0
price                     0
colour                    3
Colour_Family         13850
brand                     0
ratingCount            7740
avg_rating             7740
Occasion               1099
Print_Pattern_Type     1000
Sustainable            1302
Weave_Type             8948
Knit_or_Woven          7035
Weave_Pattern         13130
Fabric                    0
Fabric_Purity         13769
brandID                   0
dtype: int64

### Filling in remaining null values

In [113]:
sqldf("select * from dataset_final where colour is null")

,p_id,name,price,colour,Colour_Family,brand,ratingCount,avg_rating,Occasion,Print_Pattern_Type,Sustainable,Weave_Type,Knit_or_Woven,Weave_Pattern,Fabric,Fabric_Purity,brandID
0,19145038.0,Baisacrafts Women Pure Cotton Kurta with Trous...,5450.0,None,None,Baisacrafts,None,None,Festive,Geometric,Regular,Machine Weave,Woven,Regular,Pure Cotton,None,109
1,19142060.0,LIVE OK Women Boyfriend Fit High-Rise Stretcha...,1999.0,None,None,LIVE OK,None,None,Casual,None,Regular,None,None,None,Cotton,None,463
2,16124786.0,MANGO Women Hooded Sweatshirt,2390.0,None,None,MANGO,None,None,Casual,Solid,Regular,Knitted,Knitted,None,Polyester,None,498


In [114]:
avg_rating_mean = round(dataset_final["avg_rating"].mean(),6)

In [115]:
dataset_final["ratingCount"].median()

23.0

In [116]:
dataset_final["ratingCount"].mode()

0    5.0
Name: ratingCount, dtype: float64

In [117]:
ratingCount_mean = dataset_final["ratingCount"].mean()

In [118]:
colour_mode = dataset_final["colour"].mode()

In [119]:
print_pattern_mode = dataset_final["Print_Pattern_Type"].mode()
print_pattern_mode

0    Solid
Name: Print_Pattern_Type, dtype: object

In [120]:
occasion_mode = dataset_final["Occasion"].mode()

In [121]:
dataset_final.fillna({"colour":colour_mode,"avg_rating":avg_rating_mean, "ratingCount":ratingCount_mean, "Print_Pattern_Type":print_pattern_mode, "Occasion":occasion_mode}, inplace = True)
dataset_final.isna().sum()

p_id                      0
name                      0
price                     0
colour                    3
Colour_Family         13850
brand                     0
ratingCount               0
avg_rating                0
Occasion               1099
Print_Pattern_Type     1000
Sustainable            1302
Weave_Type             8948
Knit_or_Woven          7035
Weave_Pattern         13130
Fabric                    0
Fabric_Purity         13769
brandID                   0
dtype: int64

In [122]:
dataset_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14313 entries, 0 to 14312
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   p_id                14313 non-null  float64
 1   name                14313 non-null  object 
 2   price               14313 non-null  float64
 3   colour              14310 non-null  object 
 4   Colour_Family       463 non-null    object 
 5   brand               14313 non-null  object 
 6   ratingCount         14313 non-null  float64
 7   avg_rating          14313 non-null  float64
 8   Occasion            13214 non-null  object 
 9   Print_Pattern_Type  13313 non-null  object 
 10  Sustainable         13011 non-null  object 
 11  Weave_Type          5365 non-null   object 
 12  Knit_or_Woven       7278 non-null   object 
 13  Weave_Pattern       1183 non-null   object 
 14  Fabric              14313 non-null  object 
 15  Fabric_Purity       544 non-null    object 
 16  bran

>Exporting data for analysis using Apache MapReduce

In [123]:
dataset_final.to_csv("final_dataset.csv", index = False)

In [124]:
new_dataset = dataset_final.loc[:,['p_id', 'name','price','colour','brand','ratingCount','avg_rating','brand_id']]
new_dataset.to_csv("new_dataset.csv", index = False, header = False)

KeyError: "['brand_id'] not in index"

In [ ]:
dataset_final.info()

# Machine Learning

## Regression Modelling

## Pre-processing

In [ ]:
regression_data = dataset_final.loc[:,['price','avg_rating','ratingCount', 'brand_id','name','colour','brand','description','p_attributes']]

In [ ]:
le = preprocessing.LabelEncoder()

for i in regression_data.columns:
        if regression_data[i].dtype == object:
            regression_data[i] = le.fit_transform(regression_data[i])
        else:
            pass

In [ ]:
regression_data

In [ ]:
x_reg = regression_data.loc[:,['price','avg_rating','ratingCount','name','colour','description','p_attributes']]
y_reg = regression_data.loc[:,['brand_id']]

>Creating the training and testing datasets

In [ ]:
x_reg_train, x_reg_test, y_reg_train, y_reg_test = train_test_split(x_reg, y_reg, test_size=0.25)

print("Your independent training dataset contains ", x_reg_train.shape, " rows and columns.")
print("Your independent testing dataset contains ", x_reg_test.shape, " rows and columns.")
print("Your dependent training dataset contains ", y_reg_train.shape, " rows and columns.")
print("Your dependent testing dataset contains ", y_reg_test.shape, " rows and columns.")

### Linear Regression

>Training the model

In [ ]:
LR = LinearRegression()

In [ ]:
LR.fit(x_reg_train, y_reg_train)

>Testing the model

In [ ]:
LR_predict = LR.predict(x_reg_test)
LR_predict

In [ ]:
print("Linear Regressor")
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_reg_test, LR_predict))

### Bayesian Ridge Regression

In [ ]:
BayRidge = BayesianRidge()
BayRidge.fit(x_reg_train, y_reg_train)

In [ ]:
BayRidge_predict = BayRidge.predict(x_reg_test)
BayRidge_predict

In [ ]:
print("Bayesian Ridge Regressor")
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_reg_test, BayRidge_predict))

In [ ]:
dataset_final.info()

## Classification Modelling

>Pre-processing

In [ ]:
classification_data = dataset_final.loc[:,['name','colour','brand', 'brand_id','description','p_attributes']]
classification_data

In [ ]:
le = preprocessing.LabelEncoder()

for i in classification_data.columns:
        if classification_data[i].dtype == object:
            classification_data[i] = le.fit_transform(classification_data[i])
        else:
            pass

In [ ]:
classification_data

>Creating the training datasets

In [ ]:
x_class = classification_data.loc[:,['name','colour','description','p_attributes']]
y_class = classification_data.loc[:,['brand']]

In [ ]:
x_class

In [ ]:
x_class_train, x_class_test, y_class_train, y_class_test = train_test_split(x_class, y_class, test_size=0.25)

print("Your independent training dataset contains ", x_class_train.shape, " rows and columns.")
print("Your independent testing dataset contains ", x_class_test.shape, " rows and columns.")
print("Your dependent training dataset contains ", y_class_train.shape, " rows and columns.")
print("Your dependent testing dataset contains ", y_class_test.shape, " rows and columns.")

### Gaussian Naive Bayes Classifier

In [ ]:
GNB = GaussianNB()

In [ ]:
GNB.fit(x_class_train, y_class_train)

In [ ]:
GNB_predict = GNB.predict(x_class_test)

In [ ]:
print("Gaussian Naive Bayes Classifier")
print("Accuracy:", round((metrics.accuracy_score(y_class_test, GNB_predict))*100,2),"%")

### Decision Tree Classifier

In [ ]:
dtree = DecisionTreeClassifier()
dtree = dtree.fit(x_class_train, y_class_train)

In [ ]:
dtree_predict = dtree.predict(x_class_test)

In [ ]:
print("Decision Classifier")
print("Accuracy:", round((metrics.accuracy_score(y_class_test, dtree_predict))*100,2),"%")